In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime
import math

import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
from data_utils import filter_outliers

os.chdir(r'D:\Projects\MobilityForecast\Repo\mobilityforecast\data')

In [ ]:
## Read the data -> Sort by time -> filter out the data (irrleavant) -> ST feature map -> 
## ST map should capture the demand b/w pick up zone and drop off zone as a function of time(hourly)

## The idea is for the Variantional Auto encoder to predict the t+1 ST map given 1,2,3..,t-1,t ST map

## The processing of the citi bike and the weather data should also follow the same format..

In [ ]:
df = pd.read_parquet('yellow_tripdata_2022-01.parquet',engine='pyarrow')
# sorting the values is important to capture the temporal relation of the Spatio temporal data
df = df.sort_values(by='tpep_pickup_datetime', ascending=True)

In [ ]:
df['tpep_pickup_datetime']=pd.to_datetime(df['tpep_pickup_datetime']) # converting the date-time to python date-time objects to access built-in methods
df['tpep_dropoff_datetime']=pd.to_datetime(df['tpep_dropoff_datetime'])

In [ ]:
df = df.drop(columns=['store_and_fwd_flag', 'VendorID' , 'RatecodeID', 'payment_type','fare_amount', 'extra' , 'mta_tax', 'tip_amount',
'tolls_amount', 'improvement_surcharge','total_amount', 'congestion_surcharge', 'airport_fee', 'trip_distance', 'passenger_count']) # dropping the store and forward flag, VendorID is the company that provided the record(this does not add any value to our study)
df = df.dropna(axis=0) # RatecodeID and payment_type are categorical features affecting the price of the trip, we discard this for the moment as this does not add value to our study. 
df.head(5)

In [ ]:
#df = filter_outliers(df)

In [ ]:
# os.chdir(r'D:\Projects\MobilityForecast\Repo\mobilityforecast\help\images')
# BB = (-74.5, -72.8, 40.5, 41.8)
# nyc_map = plt.imread('nyc_-74.3_-73.7_40.5_40.9.png')

# # load extra image to zoom in on NYC
# BB_zoom = (-74.3, -73.7, 40.5, 40.9)
# nyc_map_zoom = plt.imread('nyc_-74.5_-72.8_40.5_41.8.png')

In [ ]:
# splitting the date-time objects to date and time

df['pickup_year'] = df['tpep_pickup_datetime'].apply(lambda t: t.year)
df['pickup_month'] = df['tpep_pickup_datetime'].apply(lambda t: t.month)
df['pickup_weekday'] = df['tpep_pickup_datetime'].apply(lambda t: t.weekday())
df['pickup_hour'] = df['tpep_pickup_datetime'].apply(lambda t: t.hour)

# df['dropoff_year'] = df['tpep_dropoff_datetime'].apply(lambda t: t.year)
# df['dropoff_month'] = df['tpep_dropoff_datetime'].apply(lambda t: t.month)
# df['dropoff_weekday'] = df['tpep_dropoff_datetime'].apply(lambda t: t.weekday())
# df['dropoff_hour'] = df['tpep_dropoff_datetime'].apply(lambda t: t.hour)

df = df.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [ ]:
# filter out the dates which are irrelavnt to the study

df = df[(df.pickup_year == 2022)]
df = df[(df.pickup_month == 1)]

In [ ]:
#df = df.drop(columns=['dropoff_year', 'dropoff_month','dropoff_weekday','dropoff_hour', 'pickup_year', 'pickup_month'])
# df = df.drop(columns=['pickup_year', 'pickup_month'])
# df.head()

In [ ]:
#features = df.groupby(['PULocationID', 'DOLocationID']).size().reset_index(name='demand')

In [ ]:
#features = features.dropna()
#features

In [ ]:
# plt.Figure(figsize=(15,15))
# table = features.pivot('PULocationID', 'DOLocationID', 'demand')
# ax = sns.heatmap(table)
# ax.invert_yaxis()
# plt.show()

In [ ]:
d